In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def read_meta(path):
    df = pd.read_csv(path)
    df['ms'] = pd.TimedeltaIndex(df['0'])
    
    return df.drop(columns='0')

In [3]:
meta = read_meta('./data/S1-ADL4-META.csv')

In [5]:
meta['activity'].value_counts()

Null                24723
Drink from Cup       2898
Close Fridge         2439
Open Fridge          2391
Open Door 1           600
Open Door 2           524
Open Dishwasher       495
Close Dishwasher      460
Close Door 1          439
Close Door 2          402
Close Drawer 2        319
Toggle Switch         287
Open Drawer 3         231
Close Drawer 3        206
Open Drawer 2         203
Clean Table           196
Close Drawer 1         63
Open Drawer 1          50
Name: activity, dtype: int64

In [7]:
client_optimizer_fn = partial(
    interpolation.__client_optimizer_fn,
    learning_rate=0.001
)

In [8]:
nest_asyncio.apply()

In [9]:
iterator = process.iterator(
    interpolation.__coefficient_fn,
    model_fn,
    client_state_fn,
    interpolation.__server_optimizer_fn,
    client_optimizer_fn
)

In [10]:
state = iterator.initialize()

In [13]:
checkpoint_manager = tff.simulation.FileCheckpointManager('./checkpoints/interpolation')

In [14]:
checkpoint_manager.save_checkpoint([client_states, state], 0)

INFO:tensorflow:Assets written to: ./checkpoints/interpolation/.temp_ckpt_0/assets


INFO:tensorflow:Assets written to: ./checkpoints/interpolation/.temp_ckpt_0/assets
